### 📚 사전 학습 모델

- `torchvision.models as models`를 이용해 ImageNet 데이터로 학습된 사전 학습 모델 (pretrained model) 사용 가능
- 또한 모델 내부의 변수명을 확인하고 모델 일부 수정 가능

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
alexnet = models.alexnet().to(device)
resnet18 = models.resnet18().to(device)
vgg16 = models.vgg16().to(device)
densenet = models.densenet161().to(device)
inception = models.inception_v3().to(device)
googlenet = models.googlenet().to(device)
shufflenet = models.shufflenet_v2_x1_0().to(device)
mobilenet_v2 = models.mobilenet_v2().to(device)
resnext50_32x4d = models.resnext50_32x4d().to(device)
wide_resnet50_2 = models.wide_resnet50_2().to(device)
mnasnet = models.mnasnet1_0().to(device)

/home/user/anaconda3/lib/python3.8/site-packages/torchvision/models/inception.py:75: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of inception_v3 will be changed in future releases of '


#### 💡 CIFAR10을 위한 ResNet18 불러오기

- 무조건 바꿔야 할 것 : 맨 처음(필터 사이즈)와 맨 끝(출력 사이즈)
- 중간중간에는 선택

In [7]:
model = torchvision.models.resnet18(pretrained=True)
# 필터 사이즈 7x7 -> 3x3
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
# 마지막 출력 노드가 1000이기 때문에 데이터에 맞춰 10으로 변경
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)

---------------------------------------------------------------------------------

### 📚 모델 프리징

- **기존의 일부 모델 변수들을 그대로 사용하기 위해 업데이트가 되지 않도록** 하는 방법
- 사전 학습된 변수를 그대로 유지하기 때문에 학습 속도와 정확도를 향상시킬 수 있고, 다른 모델과 붙여 다른 구조를 만들 수 있다.
    - ex) CNN에서 feature extraction 부분은 freezing을 시키고, classification 부분만 학습을 하기도 하는데, model tuning/freezing은 정답이 없으므로 다양한 시도가 중요하다. 

#### 💡 모델 불러오기

In [11]:
# 모델을 불러온 후, output layer의 노드를 클래스 수와 맞춘다. 
model = torchvision.models.alexnet(pretrained=True)
num_ftrs = model.classifier[6].in_features # 4096
model.classifier[6] = nn.Linear(num_ftrs,10)
model = model.to(device)

In [10]:
model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

#### 💡 모델 파라미터명 확인하기

In [18]:
for i, (name, param) in enumerate(model.named_parameters()) :
    print(i, name)

0 features.0.weight
1 features.0.bias
2 features.3.weight
3 features.3.bias
4 features.6.weight
5 features.6.bias
6 features.8.weight
7 features.8.bias
8 features.10.weight
9 features.10.bias
10 classifier.1.weight
11 classifier.1.bias
12 classifier.4.weight
13 classifier.4.bias
14 classifier.6.weight
15 classifier.6.bias


#### 💡 변수 프리징하기

In [31]:
# requires_grad를 False로 하여 학습 시 업데이트가 되지 않게 한다.
for i, (name, param) in enumerate(model.named_parameters()) :
    param.requires_grad = False
    # 합성곱 층에 대한 가중치와 편향(9번까지만)만 프리징 되면 for문을 멈춘다.
    if i == 9 :
        print('end')
        break

end


#### 💡 Requires_grad 확인하기

In [32]:
f_list = [0, 3, 6, 8, 10]
c_list = [1, 4, 6]
for i in f_list :
    print(model.features[i].weight.requires_grad)
    print(model.features[i].bias.requires_grad)
    
for j in c_list : 
    print(model.classifier[j].weight.requires_grad)
    print(model.classifier[j].bias.requires_grad)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
False
